In [21]:
import argparse

import numpy as np
import pandas as pd
import scipy.sparse as sp
import torch
from tqdm import tqdm
from sklearn.model_selection import KFold

In [9]:
%load_ext autoreload
%autoreload 2

from load_data import load_data
from model import GModel
from myutils import roc_auc, translate_result
from optimizer import Optimizer
from sampler import NewSampler

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
class Args:
    def __init__(self):
        self.device = "cpu"  # cuda:number or cpu
        self.data = "nci"  # Dataset{gdsc or ccle}


args = Args()
res, drug_finger, exprs, mut, cna, null_mask, pos_num = load_data(args)

load nci


In [22]:
cell_sum = np.sum(res, axis=1)
drug_sum = np.sum(res, axis=0)

target_dim = [
    0,  # Cell
    # 1  # Drug
]
device=torch.device(
            "cuda" if torch.cuda.is_available() else "cpu"
        )

In [26]:
def mofgcn_new(
    exprs,
    cna,
    mut,
    drug_finger,
    res_mat,
    null_mask,
    target_dim,
    target_index,
    evaluate_fun,
    args,
    seed,
    device
):
    sampler = NewSampler(res_mat, null_mask, target_dim, target_index, seed)
    model = GModel(
        adj_mat=sampler.train_data,
        gene=exprs,
        cna=cna,
        mutation=mut,
        sigma=2,
        k=11,
        iterates=3,
        feature_drug=drug_finger,
        n_hid1=192,
        n_hid2=36,
        alpha=5.74,
        device=device,
    )
    opt = Optimizer(
        model,
        sampler.train_data,
        sampler.test_data,
        sampler.test_mask,
        sampler.train_mask,
        roc_auc,
        lr=5e-4,
        epochs=1000,
        device=device,
    ).to(device)
    epoch, true_data, predict_data = opt()
    return true_data, predict_data

In [28]:
n_kfold = 1
true_data_s = pd.DataFrame()
predict_data_s = pd.DataFrame()
for dim in target_dim:
    for seed, target_index in enumerate(tqdm(np.arange(res.shape[dim]))):
        if dim:
            if drug_sum[target_index] < 10:
                continue
        else:
            if cell_sum[target_index] < 10:
                continue
        epochs = []
        for fold in range(n_kfold):
            true_data, predict_data = mofgcn_new(
                exprs=exprs,
                cna=cna,
                mut=mut,
                drug_finger=drug_finger,
                res_mat=res,
                null_mask=null_mask,
                target_dim=dim,
                target_index=target_index,
                evaluate_fun=roc_auc,
                args=args,
                seed=seed,
                device=device
            )

        true_data_s = pd.concat(
            [true_data_s, translate_result(true_data)], ignore_index=True
        )
        predict_data_s = pd.concat(
            [predict_data_s, translate_result(predict_data)], ignore_index=True
        )

  0%|          | 0/60 [00:00<?, ?it/s]

epoch:   0 loss:0.760561 auc:0.4594
epoch:  20 loss:0.563915 auc:0.6268
epoch:  40 loss:0.429277 auc:0.5320
epoch:  60 loss:0.361223 auc:0.5794
epoch:  80 loss:0.336231 auc:0.5927
epoch: 100 loss:0.325088 auc:0.5946
epoch: 120 loss:0.319036 auc:0.5937
epoch: 140 loss:0.315296 auc:0.5939
epoch: 160 loss:0.312801 auc:0.5944


  2%|▏         | 1/60 [00:02<02:03,  2.10s/it]

epoch: 180 loss:0.311081 auc:0.5945
Fit finished.
epoch:   0 loss:0.769142 auc:0.5153
epoch:  20 loss:0.564564 auc:0.5134
epoch:  40 loss:0.447301 auc:0.5287
epoch:  60 loss:0.376407 auc:0.5251
epoch:  80 loss:0.345386 auc:0.5289
epoch: 100 loss:0.330901 auc:0.5273
epoch: 120 loss:0.322780 auc:0.5280
epoch: 140 loss:0.317713 auc:0.5277
epoch: 160 loss:0.314383 auc:0.5260


  3%|▎         | 2/60 [00:04<02:10,  2.26s/it]

...

epoch: 440 loss:0.306837 auc:0.8096
Fit finished.
epoch:   0 loss:0.814019 auc:0.5026
epoch:  20 loss:0.594926 auc:0.6569
epoch:  40 loss:0.503303 auc:0.5720
epoch:  60 loss:0.416907 auc:0.5629
epoch:  80 loss:0.369802 auc:0.5865
epoch: 100 loss:0.345203 auc:0.6191
epoch: 120 loss:0.332156 auc:0.6292
epoch: 140 loss:0.324277 auc:0.6337


 98%|█████████▊| 59/60 [03:50<00:03,  4.00s/it]

epoch: 160 loss:0.319217 auc:0.6321
epoch: 180 loss:0.315849 auc:0.6304
Fit finished.
epoch:   0 loss:0.859999 auc:0.5298
epoch:  20 loss:0.598566 auc:0.5176
epoch:  40 loss:0.531452 auc:0.5549
epoch:  60 loss:0.432675 auc:0.6100
epoch:  80 loss:0.375481 auc:0.6235
epoch: 100 loss:0.347837 auc:0.6355
epoch: 120 loss:0.333810 auc:0.6386
epoch: 140 loss:0.325830 auc:0.6418
epoch: 160 loss:0.320709 auc:0.6444
epoch: 180 loss:0.317187 auc:0.6456
epoch: 200 loss:0.314670 auc:0.6457
epoch: 220 loss:0.312830 auc:0.6454
epoch: 240 loss:0.311451 auc:0.6443
epoch: 260 loss:0.310395 auc:0.6429


100%|██████████| 60/60 [03:54<00:00,  3.91s/it]

epoch: 280 loss:0.309572 auc:0.6414
epoch: 300 loss:0.308919 auc:0.6405
Fit finished.


In [29]:
true_data_s.to_csv(f"new_cell_true_{args.data}.csv")
predict_data_s.to_csv(f"new_cell_pred_{args.data}.csv")